<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Lesson-Structure" data-toc-modified-id="Lesson-Structure-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Lesson Structure</a></span><ul class="toc-item"><li><span><a href="#Questions" data-toc-modified-id="Questions-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Questions</a></span></li><li><span><a href="#Outcomes" data-toc-modified-id="Outcomes-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Outcomes</a></span></li><li><span><a href="#Objectives" data-toc-modified-id="Objectives-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Objectives</a></span></li><li><span><a href="#Outline" data-toc-modified-id="Outline-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Outline</a></span></li></ul></li><li><span><a href="#Introduction-SQL---{Victor}" data-toc-modified-id="Introduction-SQL---{Victor}-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Introduction SQL - {Victor}</a></span></li><li><span><a href="#Demonstrating-SQLite-&amp;-Python---{Jeff}" data-toc-modified-id="Demonstrating-SQLite-&amp;-Python---{Jeff}-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Demonstrating SQLite &amp; Python - {Jeff}</a></span><ul class="toc-item"><li><span><a href="#Connecting-to-our-sqlite-db-using-sqlite3" data-toc-modified-id="Connecting-to-our-sqlite-db-using-sqlite3-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Connecting to our sqlite db using sqlite3</a></span></li><li><span><a href="#Listing-the-tables-in-our-db" data-toc-modified-id="Listing-the-tables-in-our-db-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Listing the tables in our db</a></span></li><li><span><a href="#Now--select-everything-from-the-employees-table-just-to-get-a-feel-for-it" data-toc-modified-id="Now--select-everything-from-the-employees-table-just-to-get-a-feel-for-it-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Now  select everything from the employees table just to get a feel for it</a></span></li></ul></li><li><span><a href="#More-Complexity---{Victor}" data-toc-modified-id="More-Complexity---{Victor}-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>More Complexity - {Victor}</a></span><ul class="toc-item"><li><span><a href="#(3-min)-What-are-the-pros-and-cons-of-loading-a-sql-query-into-a-dataframe?" data-toc-modified-id="(3-min)-What-are-the-pros-and-cons-of-loading-a-sql-query-into-a-dataframe?-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>(3 min) What are the pros and cons of loading a sql query into a dataframe?</a></span></li><li><span><a href="#TODO:-REPLACE-←-Function-to-get-results" data-toc-modified-id="TODO:-REPLACE-←-Function-to-get-results-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>TODO: REPLACE ← Function to get results</a></span></li><li><span><a href="#TODO:--More-complex-queries" data-toc-modified-id="TODO:--More-complex-queries-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>TODO:  More complex queries</a></span></li><li><span><a href="#TODO:-Queries-with-joins" data-toc-modified-id="TODO:-Queries-with-joins-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>TODO: Queries with joins</a></span></li></ul></li><li><span><a href="#Exercises" data-toc-modified-id="Exercises-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Exercises</a></span><ul class="toc-item"><li><span><a href="#Query-Exercise-1---{Jeff}" data-toc-modified-id="Query-Exercise-1---{Jeff}-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Query Exercise 1 - {Jeff}</a></span></li><li><span><a href="#Query-Exercise-2---{Victor}" data-toc-modified-id="Query-Exercise-2---{Victor}-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Query Exercise 2 - {Victor}</a></span></li><li><span><a href="#Query-Exercise-3---{Jeff}---Students-might-do-on-their-own" data-toc-modified-id="Query-Exercise-3---{Jeff}---Students-might-do-on-their-own-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Query Exercise 3 - {Jeff} - Students might do on their own</a></span></li><li><span><a href="#Query-Exercise-4" data-toc-modified-id="Query-Exercise-4-5.4"><span class="toc-item-num">5.4&nbsp;&nbsp;</span>Query Exercise 4</a></span></li><li><span><a href="#Query-Exercise-5" data-toc-modified-id="Query-Exercise-5-5.5"><span class="toc-item-num">5.5&nbsp;&nbsp;</span>Query Exercise 5</a></span></li><li><span><a href="#Great!-It's-working.-Let's-add-some-group-by-to-remove-duplicates!" data-toc-modified-id="Great!-It's-working.-Let's-add-some-group-by-to-remove-duplicates!-5.6"><span class="toc-item-num">5.6&nbsp;&nbsp;</span>Great! It's working. Let's add some group by to remove duplicates!</a></span></li></ul></li></ul></div>

# Lesson Structure

## Questions

## Outcomes

YWBAT
- build functions to perform queries
- execute a query using multiple joins

## Objectives

- create a Pandas DataFrame from a query using sqlite3
- compare and contrast using a dataframe vs using the results from a query
- write a single join query
- write a multiple join query

## Outline

* Take Questions
* Intro to SQL
* Connect to our sqlite db
* Discussion on reading SQL database as a dataframe
* More complex queries (joins)
* Practice various queries
* Complete a query with a join
* Complete a query using multiple joins
* Wrap up

# Introduction SQL - {Victor}

[An Introduction to SQL](intro_to_sql.ipynb)

# Demonstrating SQLite & Python - {Jeff}

In [1]:
import pandas as pd
import sqlite3

## Connecting to our sqlite db using sqlite3

In [2]:
conn = sqlite3.connect('data.sqlite')
cursor = conn.cursor()

## Listing the tables in our db

In [3]:
# table_names to be a list of my table_names
query = "SELECT name FROM sqlite_master WHERE type='table';"
res = cursor.execute(query).fetchall()
print(res) # Notice these are tuples, let's extract only the names in the next cell

[('orderdetails',), ('payments',), ('offices',), ('customers',), ('orders',), ('productlines',), ('products',), ('employees',), ('contacts',), ('contacts2',)]


In [4]:
table_names = [r[0] for r in res]
table_names

['orderdetails',
 'payments',
 'offices',
 'customers',
 'orders',
 'productlines',
 'products',
 'employees',
 'contacts',
 'contacts2']

## Now  select everything from the employees table just to get a feel for it

In [5]:
query = 'select * from employees;'
res = cursor.execute(query).fetchall()
res[:2]

[('1002',
  'Murphy',
  'Diane',
  'x5800',
  'dmurphy@classicmodelcars.com',
  '1',
  '',
  'President'),
 ('1056',
  'Patterson',
  'Mary',
  'x4611',
  'mpatterso@classicmodelcars.com',
  '1',
  '1002',
  'VP Sales')]

# More Complexity - {Victor}

## (3 min) What are the pros and cons of loading a sql query into a dataframe?

- Pros
    - 

- Cons
    -  

## TODO: REPLACE ← Function to get results

## TODO:  More complex queries

##  TODO: Queries with joins

# Exercises

We'll walk through some of these and aim to have you all (students) to do them yourself!

## Query Exercise 1 - {Jeff}

Create the following table using the customers table and the orders table

<img src="images/query1.png" width="500"/>

In [ ]:
# First look at the customers dataframe 


# Then look at the orders dataframe

<details>
    <summary>Query 1 - Solution</summary>
    Query:
    
```SQL
SELECT c.customerName, c.phone, c.state, o.orderDate FROM
customers as c
JOIN orders as o using (customerNumber);
```
</details>

In [ ]:
query = """SELECT
        """

In [ ]:
join_1 = pd.read_sql(query, conn)
join_1.head()

## Query Exercise 2 - {Victor}

Create a query that results in a table with the every customer number for an employee along with the employee name and number.

Ex:

<img src="images/query2.png" width="500"/>

In [ ]:
# Work here

In [ ]:
query = """
        SELECT
        """

pd.read_sql(query, conn).head()

<details>
    <summary> Query 2 - Solution</summary>
    Query:
    
```SQL
select e.firstname, e.lastname, e.employeenumber, c.customerNumber
from employees as e
join customers as c
on e.employeenumber = c.salesrepemployeenumber;
```
    
</details>

In [ ]:
# Now let's take this and incorporate the next table orders
orders_df.head(1)

## Query Exercise 3 - {Jeff} - Students might do on their own

Create a query that results in a table with every customer name, order date, where status is disputed (you will need the customers and orders table to complete this task)

Ex:

<img src="images/query_3.png" width="500"/>

In [ ]:
# Work here

In [ ]:
query = """
        SELECT
        """

pd.read_sql(query, conn).head()

<details>
    <summary> Query 3 - Solution</summary>
    Query:
    
```SQL
SELECT c.customerName, o.status, o.orderDate
FROM customers c
JOIN orders o
USING (customerNumber)
WHERE status = "Disputed"
```
    
</details>

## Query Exercise 4

Write a query that results in the following:

Employee First and Last Name, order number and each product code for that order. 

*There should be a row for each product code*

Screenshot:

<img src="images/query4.png" width="500"/>

In [ ]:
# work here
query = """
        select
        """

pd.read_sql(query, conn).head(2)

<details>
    <summary>Query 4 Solution</summary>
    
    Query:
    
```SQL
    select e.firstname, e.lastname, o.orderNumber, od.productCode
    from employees as e
    join customers as c
    on e.employeenumber = c.salesrepemployeenumber
    join orders as o
    on o.customerNumber = c.customerNumber
    join orderdetails as od
    on od.orderNumber = o.orderNumber;
```
        
</details>

## Query Exercise 5

Write a query that results in the following:

Employee First and Last Name and each product name they sold, add an alias to match the image below

*There should be a row for each product name*

Screenshot:

<img src="images/query5.png" width="500"/>

In [ ]:
# work here
query = """
        select
        """

pd.read_sql(query, conn).head(2)

<details>
    <summary>Query 5 Solution</summary>
    
    Query:
    
```SQL
select e.firstname, e.lastname, p.productName
from employees as e
join customers as c
on e.employeenumber = c.salesrepemployeenumber
join orders as o
on o.customerNumber = c.customerNumber
join orderdetails as od
on od.orderNumber = o.orderNumber
join products as p
on p.productCode = od.productCode;
```
        
</details>

## Great! It's working. Let's add some group by to remove duplicates!

**Add aliases to result in the following**

<img src="images/query5.png" width="500"/>

In [ ]:
# remember the alias' from above

query = '''SELECT ...'''

final_df = pd.read_sql(query, conn)
final_df.head(2)

<details>
<summary>Final Query Solution</summary>

    Query:
    
```SQL
select e.firstname as fn, e.lastname as ln, p.productName as pn
from employees as e
join customers as c
on e.employeenumber = c.salesrepemployeenumber
join orders as o
on o.customerNumber = c.customerNumber
join orderdetails as od
on od.orderNumber = o.orderNumber
join products as p
on p.productCode = od.productCode
group by fn, ln, pn;

```

</details>